In [30]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import tensorflow as tf
import keras
from keras import models, layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [31]:
def parse_trackpoint(line):
    # Extract components from the line
    time_utc = line[1:7]  # HHMMSS
    latitude_raw = line[7:15]  # DDMMmmmN
    longitude_raw = line[15:24]  # DDDMMmmmE
    gps_altitude = int(line[25:30])  # GGGG
    pressure_altitude = int(line[30:35])  # LLLL

    # Convert UTC time
    hours = int(time_utc[:2])
    minutes = int(time_utc[2:4])
    seconds = int(time_utc[4:6])

    # Convert latitude to decimal degrees
    latitude_deg = int(latitude_raw[:2])
    latitude_min = float(latitude_raw[2:7]) / 1000
    latitude = latitude_deg + latitude_min / 60
    if latitude_raw[7] == 'S':
        latitude *= -1

    # Convert longitude to decimal degrees
    longitude_deg = int(longitude_raw[:3])
    longitude_min = float(longitude_raw[3:8]) / 1000
    longitude = longitude_deg + longitude_min / 60
    if longitude_raw[8] == 'W':
        longitude *= -1

    return {
        "time": f"{hours:02}:{minutes:02}:{seconds:02}",
        "coordinate": (latitude, longitude),
        "gps_altitude_m": gps_altitude,
        "pressure_altitude_m": pressure_altitude,
    }


In [32]:
file_name = "data/2024-08-03 09_11_21.igc"

with open(file_name, 'r') as file:
    lines = file.readlines()

flight_data = [line for line in lines if line.startswith('B')]
df = pd.DataFrame([parse_trackpoint(line) for line in flight_data])
del lines, flight_data
df.head()

,time,coordinate,gps_altitude_m,pressure_altitude_m
0,09:11:21,"(40.03145, 32.32855)",1142,1147
1,09:11:22,"(40.03145, 32.32855)",1142,1147
2,09:11:23,"(40.03145, 32.32855)",1142,1147
3,09:11:24,"(40.03145, 32.32855)",1142,1147
4,09:11:25,"(40.03143333333333, 32.32855)",1142,1147


In [33]:
for i in range(1, len(df)-1):
    if i % 10 > 0:
        df = df.drop([i])

flight_loginterval = int(df["time"].iloc[1][-2:]) - \
    int(df["time"].iloc[0][-2:])


def calculate_distance(row):
    if row["previus_coordinate"] is None or row["previus_coordinate"] == 0:
        return 0
    return geodesic(row["coordinate"], row["previus_coordinate"]).meters


prev_total_distance = 0


def caluculate_total_distance(row):
    global prev_total_distance
    prev_total_distance += row["distance_m"]
    return prev_total_distance

In [34]:

df["previus_coordinate"] = df["coordinate"].shift(1)
df.loc[0, "previus_coordinate"] = 0
df["distance_m"] = df.apply(calculate_distance, axis=1)
df["total_distance_m"] = df.apply(
    lambda row: caluculate_total_distance(row), axis=1)
df["climb_m"] = df["gps_altitude_m"].diff()
df["climb_rate_m/s"] = df["climb_m"] / flight_loginterval
df["speed_km/s"] = (df["distance_m"]/1000) / (flight_loginterval/3600)
df["distance_from_start_m"] = df.apply(lambda row: geodesic(
    df["coordinate"].iloc[0], row["coordinate"]).meters, axis=1)
df.to_csv("flight_data.csv", index=False)
df.head()

,time,coordinate,gps_altitude_m,pressure_altitude_m,previus_coordinate,distance_m,total_distance_m,climb_m,climb_rate_m/s,speed_km/s,distance_from_start_m
0,09:11:21,"(40.03145, 32.32855)",1142,1147,0,0.000000,0.000000,NaN,NaN,0.000000,0.000000
10,09:11:31,"(40.03145, 32.32855)",1142,1149,"(40.03145, 32.32855)",0.000000,0.000000,0.0,0.0,0.000000,0.000000
20,09:11:41,"(40.03156666666667, 32.328266666666664)",1137,1140,"(40.03145, 32.32855)",27.434746,27.434746,-5.0,-0.5,9.876508,27.434746
30,09:11:51,"(40.0319, 32.32776666666667)",1147,1149,"(40.03156666666667, 32.328266666666664)",56.490789,83.925534,10.0,1.0,20.336684,83.468388
40,09:12:01,"(40.0327, 32.328383333333335)",1139,1141,"(40.0319, 32.32776666666667)",103.251459,187.176994,-8.0,-0.8,37.170525,139.521183


In [35]:
# Example: Predict climb rate
X = df[["distance_m", "gps_altitude_m", "speed_km/s"]].values
y = df["climb_rate_m/s"].values

# Normalize data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
model.save("climb_rate_model.h5")

Epoch 1/50


/Users/serkan/Documents/Paragliding_Flight_Analyses/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.9776 - mae: 1.1725 - val_loss: nan - val_mae: nan
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6636 - mae: 1.0560 - val_loss: nan - val_mae: nan
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8475 - mae: 1.1165 - val_loss: nan - val_mae: nan
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8682 - mae: 1.1230 - val_loss: nan - val_mae: nan
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.5738 - mae: 1.0256 - val_loss: nan - val_mae: nan
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.6213 - mae: 1.0421 - val_loss: nan - val_mae: nan
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8360 - mae: 1.1163 - val_loss: nan - val_mae: nan
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8764 - mae: 1.1383 - val_loss: nan - val_mae: nan
Epoch 9/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8119 - mae: 1.0835 - val_loss: nan - val_mae: nan
Epoch 10/50
11/11 ━━━

In [37]:

loaded_model = keras.models.load_model("climb_rate_model.h5")
df["predicted_climb_rate_m/s"] = loaded_model.predict(X)

TypeError: Could not locate function 'mse'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mse', 'registered_name': 'mse'}